# Lab 1: CNN Robustness Report - SOLUTION NOTEBOOK

## Objective
Build a comprehensive CNN robustness report by:
1. Training a baseline CNN model
2. Detecting overfitting from training curves
3. Applying 2 regularization techniques and comparing
4. Running adversarial attacks (FGSM)
5. Generating a final robustness report

## Complete Implementation
This notebook contains the full solution with detailed explanations.

## Section 1: Setup & Data Loading

In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
from tqdm import tqdm

np.random.seed(42)
torch.manual_seed(42)

print("✅ Libraries imported")

In [ ]:
# Device and paths
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

DATASET_PATH = r"C:\Users\Lucifer\python_workspace\BITS\AI_Quality_Engineering\dataset"
TRAIN_PATH = os.path.join(DATASET_PATH, "train")
VAL_PATH = os.path.join(DATASET_PATH, "val")
TEST_PATH = os.path.join(DATASET_PATH, "test")

In [ ]:
# Data transformations and loading
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = ImageFolder(TRAIN_PATH, transform=transform)
val_dataset = ImageFolder(VAL_PATH, transform=transform)
test_dataset = ImageFolder(TEST_PATH, transform=transform)

class_names = train_dataset.classes
num_classes = len(class_names)

print(f"Classes: {class_names}")
print(f"Train: {len(train_dataset)} | Val: {len(val_dataset)} | Test: {len(test_dataset)}")

In [ ]:
# Create data loaders
BATCH_SIZE = 64
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print(f"✅ Data loaders created with batch size {BATCH_SIZE}")

## Section 2: Define Models with Regularization

In [ ]:
# Baseline model (no regularization)
class BaselineModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.resnet = resnet18(pretrained=False)  # Train from scratch to show overfitting
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)
    
    def forward(self, x):
        return self.resnet(x)

print("✅ BaselineModel defined")

In [ ]:
# L2 Regularized Model (identical architecture, but trained with weight decay)
class L2RegularizedModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.resnet = resnet18(pretrained=False)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)
    
    def forward(self, x):
        return self.resnet(x)

print("✅ L2RegularizedModel defined")

In [ ]:
# Dropout Model (adds dropout layer before classification)
class DropoutModel(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.5):
        super().__init__()
        self.resnet = resnet18(pretrained=False)
        # Replace final layer with Dropout + Linear
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Dropout(dropout_rate),  # Randomly drops 50% of neurons
            nn.Linear(in_features, num_classes)
        )
    
    def forward(self, x):
        return self.resnet(x)

print("✅ DropoutModel defined")

## Section 3: Training & Evaluation Functions

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Track metrics
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    avg_loss = running_loss / len(loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

print("✅ train_epoch() defined")

In [ ]:
def evaluate(model, loader, criterion, device):
    """Evaluate on validation or test set"""
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_loss = running_loss / len(loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

print("✅ evaluate() defined")

## Section 4: Train All Three Models

In [ ]:
# Train Baseline Model (No Regularization)
print("\n" + "="*60)
print("Training BASELINE Model (No Regularization)")
print("="*60)

baseline_model = BaselineModel(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer_baseline = optim.Adam(baseline_model.parameters(), lr=0.001, weight_decay=0)  # No weight decay

baseline_history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
NUM_EPOCHS = 30

for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train_epoch(baseline_model, train_loader, criterion, optimizer_baseline, device)
    val_loss, val_acc = evaluate(baseline_model, val_loader, criterion, device)
    
    baseline_history['train_loss'].append(train_loss)
    baseline_history['train_acc'].append(train_acc)
    baseline_history['val_loss'].append(val_loss)
    baseline_history['val_acc'].append(val_acc)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

print("✅ Baseline model trained")

In [ ]:
# Train L2 Regularized Model
print("\n" + "="*60)
print("Training L2 REGULARIZED Model (Weight Decay = 0.001)")
print("="*60)

l2_model = L2RegularizedModel(num_classes).to(device)
optimizer_l2 = optim.Adam(l2_model.parameters(), lr=0.001, weight_decay=0.001)  # L2 regularization

l2_history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train_epoch(l2_model, train_loader, criterion, optimizer_l2, device)
    val_loss, val_acc = evaluate(l2_model, val_loader, criterion, device)
    
    l2_history['train_loss'].append(train_loss)
    l2_history['train_acc'].append(train_acc)
    l2_history['val_loss'].append(val_loss)
    l2_history['val_acc'].append(val_acc)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

print("✅ L2 Regularized model trained")

In [ ]:
# Train Dropout Model
print("\n" + "="*60)
print("Training DROPOUT Model (Dropout Rate = 0.5)")
print("="*60)

dropout_model = DropoutModel(num_classes, dropout_rate=0.5).to(device)
optimizer_dropout = optim.Adam(dropout_model.parameters(), lr=0.001, weight_decay=0)

dropout_history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train_epoch(dropout_model, train_loader, criterion, optimizer_dropout, device)
    val_loss, val_acc = evaluate(dropout_model, val_loader, criterion, device)
    
    dropout_history['train_loss'].append(train_loss)
    dropout_history['train_acc'].append(train_acc)
    dropout_history['val_loss'].append(val_loss)
    dropout_history['val_acc'].append(val_acc)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

print("✅ Dropout model trained")

## Section 5: Detect Overfitting

In [ ]:
# Plot overfitting analysis
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
fig.suptitle('Overfitting Analysis: Baseline vs L2 vs Dropout', fontsize=16, fontweight='bold')

models_data = [
    ('Baseline\n(No Regularization)', baseline_history, axes[0]),
    ('L2 Regularized\n(Weight Decay)', l2_history, axes[1]),
    ('Dropout\n(Dropout Rate=0.5)', dropout_history, axes[2])
]

for model_name, history, ax in models_data:
    ax.plot(history['train_acc'], label='Train Accuracy', marker='o', linewidth=2, markersize=4)
    ax.plot(history['val_acc'], label='Val Accuracy', marker='s', linewidth=2, markersize=4)
    ax.set_xlabel('Epoch', fontsize=11)
    ax.set_ylabel('Accuracy (%)', fontsize=11)
    ax.set_title(model_name, fontsize=12, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.set_ylim([0, 105])

plt.tight_layout()
plt.savefig('overfitting_comparison.png', dpi=150)
plt.show()

print("✅ Overfitting curves plotted")

In [ ]:
# Calculate overfitting metrics
overfitting_analysis = {}

baseline_gap = baseline_history['train_acc'][-1] - baseline_history['val_acc'][-1]
overfitting_analysis['Baseline'] = baseline_gap

l2_gap = l2_history['train_acc'][-1] - l2_history['val_acc'][-1]
overfitting_analysis['L2 Regularized'] = l2_gap

dropout_gap = dropout_history['train_acc'][-1] - dropout_history['val_acc'][-1]
overfitting_analysis['Dropout'] = dropout_gap

print("\n📊 OVERFITTING ANALYSIS (Train-Val Gap):")
for model_name, gap in overfitting_analysis.items():
    print(f"{model_name:20s}: {gap:6.2f}% gap", end="")
    if gap > 10:
        print(" ⚠️  HIGH OVERFITTING")
    elif gap > 5:
        print(" ⚠️  MODERATE OVERFITTING")
    else:
        print(" ✅ GOOD GENERALIZATION")

## Section 6: Model Complexity Analysis

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def get_model_size_mb(model):
    torch.save(model.state_dict(), "temp_model.pth")
    size_mb = os.path.getsize("temp_model.pth") / (1024 * 1024)
    os.remove("temp_model.pth")
    return size_mb

# Calculate model sizes
model_complexity = {}

for model_name, model in [('Baseline', baseline_model), ('L2 Regularized', l2_model), ('Dropout', dropout_model)]:
    params = count_parameters(model)
    size = get_model_size_mb(model)
    model_complexity[model_name] = {'params': params, 'size': size}
    print(f"{model_name:20s}: {params:,} parameters | {size:.2f} MB")

## Section 7: Robustness Testing - Adversarial Attacks

In [ ]:
# Gaussian Noise Perturbation
def add_gaussian_noise(images, noise_std=0.1):
    """
    Add Gaussian noise to images.
    
    Args:
        images: Tensor of shape (B, C, H, W)
        noise_std: Standard deviation of Gaussian noise
    
    Returns:
        Noisy images clipped to [-1, 1] range
    """
    noise = torch.randn_like(images) * noise_std  # Create random noise
    noisy_images = images + noise  # Add noise to images
    return torch.clamp(noisy_images, -1, 1)  # Clip to valid range

print("✅ Gaussian noise function defined")

In [ ]:
# FGSM Attack
def fgsm_attack(model, images, labels, device, epsilon=0.05):
    """
    Fast Gradient Sign Method (FGSM) Attack.
    
    Creates adversarial examples by moving in the gradient direction.
    
    Args:
        model: Neural network
        images: Input images
        labels: True labels
        device: Device to run on
        epsilon: Attack strength (max perturbation per pixel)
    
    Returns:
        Adversarial images
    """
    images.requires_grad = True  # Enable gradient tracking
    
    # Forward pass
    outputs = model(images)
    loss = nn.CrossEntropyLoss()(outputs, labels)
    
    # Compute gradients
    model.zero_grad()
    loss.backward()
    
    # Get gradient sign and create adversarial examples
    data_grad = images.grad.data
    sign_data_grad = data_grad.sign()  # Sign of gradient
    
    # Perturb in gradient direction
    perturbed_images = images + epsilon * sign_data_grad
    
    # Clip to valid range
    return torch.clamp(perturbed_images, -1, 1).detach()

print("✅ FGSM attack function defined")

In [ ]:
# Evaluate on noisy images
def evaluate_on_noisy(model, loader, device, noise_std=0.1):
    """Evaluate model on Gaussian noise-perturbed images"""
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc=f"Testing noise (σ={noise_std})", leave=False):
            images, labels = images.to(device), labels.to(device)
            # Add Gaussian noise
            noisy_images = add_gaussian_noise(images, noise_std=noise_std)
            # Get predictions
            outputs = model(noisy_images)
            _, predicted = torch.max(outputs, 1)
            # Update accuracy
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    return accuracy

print("✅ Noisy evaluation function defined")

In [ ]:
# Evaluate on adversarial images
def evaluate_on_adversarial(model, loader, device, epsilon=0.05):
    """Evaluate model on FGSM adversarial examples"""
    model.eval()
    correct = 0
    total = 0
    
    for images, labels in tqdm(loader, desc=f"Testing FGSM (ε={epsilon})", leave=False):
        images, labels = images.to(device), labels.to(device)
        # Generate adversarial examples
        adv_images = fgsm_attack(model, images.clone(), labels, device, epsilon=epsilon)
        # Evaluate on adversarial images
        outputs = model(adv_images)
        _, predicted = torch.max(outputs, 1)
        # Update accuracy
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    return accuracy

print("✅ Adversarial evaluation function defined")

In [ ]:
# Test all models on clean, noisy, and adversarial data
print("\n" + "="*70)
print("ROBUSTNESS EVALUATION")
print("="*70)

robustness_results = {}

for model_name, model in [('Baseline', baseline_model), ('L2 Regularized', l2_model), ('Dropout', dropout_model)]:
    print(f"\n{model_name}:")
    
    # Clean accuracy
    clean_loss, clean_acc = evaluate(model, test_loader, criterion, device)
    # Noisy accuracy (Gaussian noise with σ=0.1)
    noisy_acc = evaluate_on_noisy(model, test_loader, device, noise_std=0.1)
    # Adversarial accuracy (FGSM with ε=0.05)
    adv_acc = evaluate_on_adversarial(model, test_loader, device, epsilon=0.05)
    
    robustness_results[model_name] = {
        'clean': clean_acc,
        'noisy': noisy_acc,
        'adversarial': adv_acc
    }
    
    print(f"  Clean Accuracy:               {clean_acc:.2f}%")
    print(f"  Noisy Accuracy (σ=0.1):        {noisy_acc:.2f}%")
    print(f"  Adversarial Accuracy (ε=0.05): {adv_acc:.2f}%")

## Section 8: Generate Final Report

In [ ]:
# Create comprehensive report
print("\n" + "="*80)
print("🎯 CNN ROBUSTNESS REPORT")
print("="*80)

print("\n📊 ACCURACY METRICS:")
print("-" * 80)
print(f"{'Model':<25} {'Clean':<15} {'Noisy':<15} {'Adversarial':<15}")
print("-" * 80)

for model_name in ['Baseline', 'L2 Regularized', 'Dropout']:
    clean = robustness_results[model_name]['clean']
    noisy = robustness_results[model_name]['noisy']
    adv = robustness_results[model_name]['adversarial']
    print(f"{model_name:<25} {clean:>6.2f}%{'':<7} {noisy:>6.2f}%{'':<7} {adv:>6.2f}%")

print("\n⏱️  INFERENCE TIME & MODEL SIZE:")
print("-" * 80)
print(f"{'Model':<25} {'Parameters':<20} {'Size (MB)':<15}")
print("-" * 80)

for model_name in ['Baseline', 'L2 Regularized', 'Dropout']:
    params = model_complexity[model_name]['params']
    size = model_complexity[model_name]['size']
    print(f"{model_name:<25} {params:>12,}{'':<6} {size:>6.2f}")

print("\n🔗 GENERALIZATION & REGULARIZATION:")
print("-" * 80)
print(f"{'Model':<25} {'Train-Val Gap':<15} {'Status':<20}")
print("-" * 80)

for model_name in ['Baseline', 'L2 Regularized', 'Dropout']:
    gap = overfitting_analysis[model_name]
    if gap > 10:
        status = "High Overfitting"
    elif gap > 5:
        status = "Moderate Overfitting"
    else:
        status = "Good Generalization"
    print(f"{model_name:<25} {gap:>6.2f}%{'':<7} {status:<20}")

In [ ]:
# Create visualization
fig, ax = plt.subplots(figsize=(12, 6))

model_names = ['Baseline', 'L2 Regularized', 'Dropout']
x = np.arange(len(model_names))
width = 0.25

clean_accs = [robustness_results[m]['clean'] for m in model_names]
noisy_accs = [robustness_results[m]['noisy'] for m in model_names]
adv_accs = [robustness_results[m]['adversarial'] for m in model_names]

ax.bar(x - width, clean_accs, width, label='Clean', alpha=0.8, color='#2ecc71')
ax.bar(x, noisy_accs, width, label='Noisy (σ=0.1)', alpha=0.8, color='#f39c12')
ax.bar(x + width, adv_accs, width, label='Adversarial (ε=0.05)', alpha=0.8, color='#e74c3c')

ax.set_xlabel('Model', fontsize=12, fontweight='bold')
ax.set_ylabel('Accuracy (%)', fontsize=12, fontweight='bold')
ax.set_title('Robustness Comparison: Clean vs Noisy vs Adversarial', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(model_names, fontsize=11)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim([0, 105])

# Add value labels on bars
for bars in [ax.patches[i::len(model_names)] for i in range(len(model_names))]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('robustness_summary.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Summary visualization created")

## Key Findings & Analysis

In [ ]:
print("\n" + "="*80)
print("💡 OBSERVATIONS & ANALYSIS")
print("="*80)

print("\n1. OVERFITTING ANALYSIS:")
print("   - The Baseline model shows the highest train-val gap (most overfitting)")
print(f"     Baseline gap: {overfitting_analysis['Baseline']:.2f}%")
print(f"     L2 Regularized gap: {overfitting_analysis['L2 Regularized']:.2f}%")
print(f"     Dropout gap: {overfitting_analysis['Dropout']:.2f}%")
print("   This is expected because we trained without any regularization.")

print("\n2. REGULARIZATION EFFECTIVENESS:")
l2_effect = overfitting_analysis['Baseline'] - overfitting_analysis['L2 Regularized']
dropout_effect = overfitting_analysis['Baseline'] - overfitting_analysis['Dropout']
print(f"   - L2 Regularization reduced overfitting gap by {l2_effect:.2f}%")
print(f"   - Dropout reduced overfitting gap by {dropout_effect:.2f}%")
if dropout_effect > l2_effect:
    print("   → Dropout is more effective at controlling overfitting")
else:
    print("   → L2 Regularization is more effective at controlling overfitting")

print("\n3. ROBUSTNESS ASSESSMENT:")
for model_name in model_names:
    clean = robustness_results[model_name]['clean']
    adv = robustness_results[model_name]['adversarial']
    drop = clean - adv
    print(f"   - {model_name}: Clean {clean:.2f}% → Adversarial {adv:.2f}% (drop: {drop:.2f}%)")

print("\n4. ACCURACY-COMPLEXITY TRADEOFF:")
for model_name in model_names:
    acc = robustness_results[model_name]['clean']
    size = model_complexity[model_name]['size']
    print(f"   - {model_name}: {acc:.2f}% accuracy, {size:.2f} MB model size")
print("   → Model size is identical across all three (same architecture)")
print("   → Performance differs due to regularization, not model complexity")

print("\n5. RECOMMENDATIONS:")
best_model = max(model_names, key=lambda m: robustness_results[m]['clean'])
best_robust = min(model_names, key=lambda m: robustness_results[m]['clean'] - robustness_results[m]['adversarial'])
print(f"   - For highest clean accuracy: {best_model}")
print(f"   - For best adversarial robustness: {best_robust}")
print(f"   - Deploy {best_robust} in production for better robustness")
print("\n" + "="*80)

## Summary Table

In [ ]:
import pandas as pd

# Create comprehensive summary table
summary_data = {
    'Model': model_names,
    'Clean Acc (%)': [robustness_results[m]['clean'] for m in model_names],
    'Noisy Acc (%)': [robustness_results[m]['noisy'] for m in model_names],
    'Adversarial Acc (%)': [robustness_results[m]['adversarial'] for m in model_names],
    'Overfitting Gap (%)': [overfitting_analysis[m] for m in model_names],
    'Model Size (MB)': [model_complexity[m]['size'] for m in model_names],
    'Parameters': [model_complexity[m]['params'] for m in model_names]
}

df = pd.DataFrame(summary_data)
print("\n📋 COMPREHENSIVE SUMMARY TABLE:")
print(df.to_string(index=False))
print("\n✅ Lab 1 Complete!")